---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv',encoding='ISO-8859-1')
train.head(3)



/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [9]:
train.shape

(250306, 34)

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ticket_id                   250306 non-null  int64  
 1   agency_name                 250306 non-null  object 
 2   inspector_name              250306 non-null  object 
 3   violator_name               250272 non-null  object 
 4   violation_street_number     250306 non-null  float64
 5   violation_street_name       250306 non-null  object 
 6   violation_zip_code          0 non-null       float64
 7   mailing_address_str_number  246704 non-null  float64
 8   mailing_address_str_name    250302 non-null  object 
 9   city                        250306 non-null  object 
 10  state                       250213 non-null  object 
 11  zip_code                    250305 non-null  object 
 12  non_us_str_code             3 non-null       object 
 13  country       

In [17]:
train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [26]:
train['state'].unique()

array(['IL', 'MI', 'CA', 'NY', 'VA', 'MN', 'NV', 'PA', 'LA', 'MD', 'FL',
       'ME', 'TN', 'KY', 'TX', 'AZ', 'OH', 'GA', 'AR', 'KS', 'MO', 'WI',
       'IN', 'MS', 'MA', 'OR', 'NJ', 'WA', 'UT', 'SC', 'AL', 'AK', 'NB',
       'DC', 'CT', 'OK', 'UK', 'NC', 'CO', 'RI', 'NM', 'VT', 'MT', 'IA',
       'ID', 'DE', 'PR', 'NH', 'VI', 'SD', 'QC', 'ON', 'HI', 'QL', nan,
       'WY', 'WV', 'ND', 'BC', 'BL'], dtype=object)

In [25]:
train['country'].unique()

array(['USA', 'Cana', 'Aust', 'Egyp', 'Germ'], dtype=object)

In [2]:
test = pd.read_csv('test.csv',encoding='ISO-8859-1')

In [9]:
test.set_index('ticket_id',inplace=True)

In [10]:
test.head(3)

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [11]:
test.columns

Index(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 284932 to 369851
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   agency_name                 61001 non-null  object 
 1   inspector_name              61001 non-null  object 
 2   violator_name               60973 non-null  object 
 3   violation_street_number     61001 non-null  float64
 4   violation_street_name       61001 non-null  object 
 5   violation_zip_code          24024 non-null  object 
 6   mailing_address_str_number  59987 non-null  object 
 7   mailing_address_str_name    60998 non-null  object 
 8   city                        61000 non-null  object 
 9   state                       60670 non-null  object 
 10  zip_code                    60998 non-null  object 
 11  non_us_str_code             0 non-null      float64
 12  country                     61001 non-null  object 
 13  ticket_issued_date       

In [26]:
test.shape

(61001, 6)

In [14]:
test.drop([ 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition',
       'clean_up_cost', 'grafitti_status'],axis=1,inplace=True)

In [7]:
train.drop(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 
       'clean_up_cost', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail'],axis=1,inplace=True)

In [8]:
train.columns

Index(['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'judgment_amount', 'compliance'],
      dtype='object')

Todos los elementos nulos de nuestro DataFrame

In [17]:
train.size-train.isnull().sum().sum()

1661715

In [18]:
train = train[pd.notnull(train.compliance)]

In [25]:
train.shape

(159880, 7)

In [23]:
missing_count = train.isnull().sum()

In [24]:
missing_count

fine_amount        0
admin_fee          0
state_fee          0
late_fee           0
discount_amount    0
judgment_amount    0
compliance         0
dtype: int64

In [22]:
train.head(3)

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,compliance
0,250.0,20.0,10.0,25.0,0.0,305.0,0.0
1,750.0,20.0,10.0,75.0,0.0,855.0,1.0
5,250.0,20.0,10.0,25.0,0.0,305.0,0.0


In [31]:
train.shape

(159880, 7)

In [35]:
features = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount']

In [ ]:
features

In [15]:
test.columns

Index(['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'judgment_amount'],
      dtype='object')

In [16]:
test.head(3)

,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount
ticket_id,,,,,,
284932,200.0,20.0,10.0,20.0,0.0,250.0
285362,1000.0,20.0,10.0,100.0,0.0,1130.0
285361,100.0,20.0,10.0,10.0,0.0,140.0


In [30]:
test.shape

(61001, 6)

In [27]:
target = train['compliance']

In [82]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score




def blight_model():
    
    # Your code here
    train = pd.read_csv('train.csv',encoding='ISO-8859-1')
    train.drop(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 
       'clean_up_cost', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail'],axis=1,inplace=True)
    
    train = train[pd.notnull(train.compliance)]
    features = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount']
    y = ['compliance']
    
    
    test = pd.read_csv('test.csv',encoding='ISO-8859-1')
    test.set_index('ticket_id',inplace=True)
    
    model = RandomForestClassifier(n_estimators=100,max_depth=5,random_state=0)
    
    X_train, X_test, y_train, y_test = train_test_split(train[features], train[y], random_state = 0)
    model.fit(train[features], train[y])
    pred = model.predict_proba(test[features])[:,1]
    test['compliance'] = pred
    
    
    return test['compliance'] # Your answer here

In [83]:
blight_model()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-82-61cc1defb3e2>:36: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train[features], train[y])


ticket_id
284932    0.063239
285362    0.024542
285361    0.064861
285338    0.063239
285346    0.064861
            ...   
376496    0.024542
376497    0.024542
376499    0.064861
376500    0.064861
369851    0.271009
Name: compliance, Length: 61001, dtype: float64

In [75]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

def blight_model():
    
    # Your code here
    train = pd.read_csv('train.csv',encoding='ISO-8859-1')
    train.drop(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 
       'clean_up_cost', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail'],axis=1,inplace=True)
    
    train = train[pd.notnull(train.compliance)]
    features = ['fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount']
    y = ['compliance']
    
    
    test = pd.read_csv('test.csv',encoding='ISO-8859-1')
    test.set_index('ticket_id',inplace=True)
    
    parameters = {'n_estimators': [5,10,15],
                 'max_depth': [5,10,15]}
    
    model = RandomForestClassifier()
    
    X_train, X_test, y_train, y_test = train_test_split(train[features], train[y], random_state = 0)
    
    # alternative metric to optimize over grid parameters: AUC
    grid_clf_auc = GridSearchCV(model, param_grid = parameters, scoring = 'roc_auc')
    grid_clf_auc.fit(train[features], train[y])
    
    model.fit(train[features], train[y])
    pred = model.predict_proba(test[features])[:,1]
    test['compliance'] = pred
    
    print('Grid best parameter (max. accuracy): ', grid_clf_auc.best_params_)
    print('Grid best score (accuracy): ', grid_clf_auc.best_score_)
    
    return test['compliance'] # Your answer here

In [77]:
#blight_model()